# Pipeline to delimit ICE

#### N.B. Charger l'enviro macsyfinder avant d'ouvrir ce notebook 
conda activate macsyfinder

jupyter-notebook Tutorial_ICE

Ce notebook de référence est tiré d'un article scientifique portant sur la détection des ICEs.

Jean Cury, Marie Touchon, Eduardo P. C. Rocha, Integrative and conjugative elements and their hosts: composition, distribution and organization, Nucleic Acids Research, Volume 45, Issue 15, 6 September 2017, Pages 8943–8956, https://doi.org/10.1093/nar/gkx607

Abstract

Conjugation of single-stranded DNA drives horizontal gene transfer between bacteria and was widely studied in conjugative plasmids. The organization and function of integrative and conjugative elements (ICE), even if they are more abundant, was only studied in a few model systems. Comparative genomics of ICE has been precluded by the difficulty in finding and delimiting these elements. Here, we present the results of a method that circumvents these problems by requiring only the identification of the conjugation genes and the species’ pan-genome. We delimited 200 ICEs and this allowed the first large-scale characterization of these elements. We quantified the presence in ICEs of a wide set of functions associated with the biology of mobile genetic elements, including some that are typically associated with plasmids, such as partition and replication. Protein sequence similarity networks and phylogenetic analyses revealed that ICEs are structured in functional modules. Integrases and conjugation systems have different evolutionary histories, even if the gene repertoires of ICEs can be grouped in function of conjugation types. Our characterization of the composition and organization of ICEs paves the way for future functional and evolutionary analyses of their cargo genes, composed of a majority of unknown function genes.

Dans le matériel supplémentaire de cet article (gkx607_Supp.zip), on peut trouver un tutoriel (jupyter-notebook) permettant de faire l'analyse des ICEs. Utilisé tel que, le tutoriel présente quelques bugs, qui empêche un utilisateur même expériementé de reproduire les résultats.

J'ai adapté le code pour l'environnement particulier de la station de travail QCSHERA673892DX. En effet, l'annotation de prokka est particulière sur cette station de travail, car on souhaite annoter les gènes de résistance aux antibiotiques avec la dernière version de la base de données CARD.


Jean-Simon Brouard le 23 décembre 2019


#### Vérification des principales dépendances
On devrait y voir roary, prokka, matplotlib, biopython, pandas, numpy

In [47]:
!conda list | grep -e biopython -e roary -e matplotlib -e prokka -e macsyfinder -e pandas -e numpy

# packages in environment at /home/brouardjs/miniconda3/envs/macsyfinder:
biopython                 1.74             py27h516909a_0    conda-forge
matplotlib                2.2.4                    py27_2    conda-forge
matplotlib-base           2.2.4            py27h250f245_2    conda-forge
numpy                     1.16.5           py27h95a1406_0    conda-forge
pandas                    0.24.2           py27he6710b0_0    anaconda
prokka                    1.14.5                  pl526_0    bioconda
roary                     3.13.0          pl526h516909a_0    bioconda


####  Il faut aussi les exécutables usearch et le programme macsyfinder

Note a moi-même, on voit qu'il y a deux dépendances qui n'ont pas ete installees dans l'environ macsyfinder, hum hum!

In [14]:
!which macsyfinder
!which usearch

/home/brouardjs/miniconda3/envs/macsyfinder/bin/macsyfinder
/home/brouardjs/bioinfo_programs/macsyfinder-1.0.5/usearch


In [15]:
#probleme de prokka

!which prokka

/home/brouardjs/miniconda3/envs/macsyfinder/bin/prokka


In this notebook, we'll find ICE and delimit them in the Haemophilus influenzae species

1. First we'll get the complete genome from NCBI
3. We'll build the core genome
2. We'll detect the conjugative system in the genomes
4. We'll identify the core-genes flanking the conjugative system (defining a spot)
5. We'll build the pan-genome of the spot
6. We'll make a visual representation of the spot with gene family frequency to delimit the ICE.

<p><div class="lev1"><a href="#Data"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data</a></div><div class="lev2"><a href="#Download-fasta-sequences"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Download fasta sequences</a></div><div class="lev2"><a href="#Make-CDS-annotation-with-Prokka"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Make CDS annotation with Prokka</a></div><div class="lev1"><a href="#Build-Core-genomes"><span class="toc-item-num">3&nbsp;&nbsp;</span>Build Core genomes</a></div><div class="lev1"><a href="#Identification-of-Conjugative-systems"><span class="toc-item-num">4&nbsp;&nbsp;</span>Identification of Conjugative systems</a></div><div class="lev1"><a href="#Identify-the-flanking-core-genes:-finding-a-spot"><span class="toc-item-num">5&nbsp;&nbsp;</span>Identify the flanking core genes: finding a spot</a></div><div class="lev1"><a href="#Pan-Genome-of-the-spot"><span class="toc-item-num">6&nbsp;&nbsp;</span>Pan-Genome of the spot</a></div><div class="lev1"><a href="#Format-data-to-plot"><span class="toc-item-num">7&nbsp;&nbsp;</span>Format data to plot</a></div><div class="lev2"><a href="#Make-a-table-for-each-ICE"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Make a table for each ICE</a></div><div class="lev1"><a href="#Plot-the-spot"><span class="toc-item-num">8&nbsp;&nbsp;</span>Plot the spot</a></div>

Approximate time ot run this notebook: ~15-20 minutes.

**NB**: The pipeline here works because the 3 ICEs we will found are in the same spot. But often, ICEs are found in different spots, and it implies to deal with that information to not mix everything.

# Data

In [16]:
mkdir Sequences

In [17]:
mkdir Sequences/Replicon

## Download fasta sequences

One needs to download the data, let's to it with python

In [18]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "jean-simon.brouard@canada.ca" #tells ncbi who you are.

I download the sequence from the NC number of 10 *Haemophilus influenzae*

In [19]:
for nc in ["NC_000907", "NC_007146", "NC_009566", "NC_009567", "NC_014920", "NC_014922", "NC_016809", "NC_017452", "NC_017451", "NC_022356"]:
    handle = Entrez.esearch(db="nucleotide", term=nc, rettype="gb", retmode="text")
    record = Entrez.read(handle)
    if len(record["IdList"]) == 0: # if no hit in genbank
        print nc
    handle2 = Entrez.efetch(db="nucleotide", id=record["IdList"][0], rettype="fasta", retmode="text")
    replicons = SeqIO.read(handle2, "fasta")
    SeqIO.write(replicons,
                "Sequences/Replicon/" + nc +".fst",
                "fasta")

In [20]:
!wc -l Sequences/Replicon/*

   30504 Sequences/Replicon/NC_000907.fst
   31910 Sequences/Replicon/NC_007146.fst
   30219 Sequences/Replicon/NC_009566.fst
   31455 Sequences/Replicon/NC_009567.fst
   33099 Sequences/Replicon/NC_014920.fst
   33452 Sequences/Replicon/NC_014922.fst
   33027 Sequences/Replicon/NC_016809.fst
   32207 Sequences/Replicon/NC_017451.fst
   30324 Sequences/Replicon/NC_017452.fst
   30938 Sequences/Replicon/NC_022356.fst
  317135 total


## Make CDS annotation with Prokka

[Prokka](https://github.com/tseemann/prokka)


#### Note importante a propos de l'annotation avec PROKKA - JSB

Puisque nous avons une version modifiee de prokka qui utilise un db locale (et CARD), il est important de lancer l'annotation avec prokka dans l'environnement conda prokka en dehors de ce notebook!








In [23]:
mkdir Sequences/Annotations

In [24]:
!pwd

/data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation


#### Donc aller dans un terminal et rouler en dehors de ce notebook

conda activate prokka

./prokka_DPL_laucher.sh

In [ ]:
# A titre informatif, voici le contenu de mon script bash

'''

#!/bin/bash
i=1
for nc in "NC_000907" "NC_007146" "NC_009566" "NC_009567" "NC_014920" "NC_014922" "NC_016809" "NC_017452" "NC_017451" "NC_022356"; do
    printf -v j "%03d" $i
    echo HAIN"$j"
    prokka --kingdom Bacteria \
            --genus Haemophilus Sequences/Replicon/"$nc".fst \
            --outdir Sequences/Annotations/HAIN"$j" \
            --proteins /data/ext4/dataDP/similarity_searches/db/card_original/protein_fasta_protein_homolog_model.fasta \
            --force \
            --locustag HAIN"$j" \
            --prefix HAIN"$j" \
            --cpus 0 >> LOG_prokka 2>&1
    i=$((i+1))
done


'''

# Build Core genomes

With [Roary](https://sanger-pathogens.github.io/Roary/)



Move all `.gff` files in the same folder so Roary can work with them

In [26]:
mkdir Sequences/GFF

In [27]:
!cp Sequences/Annotations/*/*gff Sequences/GFF/

In [28]:
mkdir Sequences/Pangenome

#### Pas de problème ici, roary est dans l'enviro macsyfinder

In [29]:
!roary -p 20 -f Sequences/Pangenome/ Sequences/GFF/*.gff > LOG_roary 2>&1

Get core-genes

**BEWARE** to change the random number generated by `roary` (here `_1494846018`)

In [31]:
!query_pan_genome -g Sequences/Pangenome/_1576775051/clustered_proteins -a intersection Sequences/GFF/*.gff

In [32]:
mv pan_genome_results core_genome.txt

In [33]:
!head core_genome.txt

gdhA: HAIN001_00203	HAIN002_00273	HAIN003_00473	HAIN004_00707	HAIN005_01972	HAIN006_01111	HAIN007_00257	HAIN008_00454	HAIN009_00399	HAIN010_00871
group_245: HAIN001_01230	HAIN002_01260	HAIN003_01252	HAIN004_01896	HAIN005_01094	HAIN006_01885	HAIN007_01365	HAIN008_01198	HAIN009_01245	HAIN010_01455
fbpA: HAIN001_00104	HAIN002_00172	HAIN003_00582	HAIN004_00596	HAIN005_00137	HAIN006_00099	HAIN007_00161	HAIN008_00553	HAIN009_00503	HAIN010_01444
group_502: HAIN001_00588	HAIN002_00655	HAIN003_00034	HAIN004_01214	HAIN005_01729	HAIN006_00655	HAIN007_00715	HAIN008_00024	HAIN009_00024	HAIN010_00204
rseB: HAIN001_00677	HAIN002_00682	HAIN003_01821	HAIN004_01240	HAIN005_01625	HAIN006_00556	HAIN007_00803	HAIN008_01742	HAIN009_01872	HAIN010_00228
menB: HAIN001_01035	HAIN002_01096	HAIN003_01426	HAIN004_01693	HAIN005_01259	HAIN006_00161	HAIN007_01162	HAIN008_01363	HAIN009_01421	HAIN010_00602
thrB: HAIN001_00094	HAIN002_00163	HAIN003_00589	HAIN004_00590	HAIN005_00131	HAIN006_00089	HAIN007_00152	HAIN008_00

In [34]:
!wc -l core_genome.txt

1084 core_genome.txt


Remove first columns

In [35]:
!awk -F":" '{print $2}' core_genome.txt > core_genome_2.txt

# Identification of Conjugative systems

I downloaded the [module CONJScan](https://minhaskamal.github.io/DownGit/#/home?url=https://github.com/gem-pasteur/Macsyfinder_models/tree/master/Data/Conjugation) with model and profiles for conjugative systems to use with macsyfinder. I put it under a Conjugation folder and rename it "Models"

In [ ]:
mkdir Conjugation

In [ ]:
ls Conjugation/

In [ ]:
# l'executable fonctionne
!/usr/local/bin/macsyfinder

In [ ]:
# Pour reference, voici le contenu de l'excutable macsyfinder_laucher.sh
'''
#!/bin/bash
for seq in Sequences/Annotations/*/*faa; do
    name=${seq##*/}
    for conj_type in typeF typeB typeC typeFATA typeFA typeG typeI typeT; do
        macsyfinder "$conj_type" \
                    -w 20 \
                    --db-type ordered_replicon \
                    -d Conjugation/Models/DEF \
                    -p Conjugation/Models/HMM \
                    --profile-suffix .hmm \
                    --sequence-db "$seq" \
                    -o Conjugation/${name%%.*}\_"$conj_type"
    done
done
'''

In [42]:
!./macsyfinder_laucher.sh

MacSyFinder's results will be stored in Conjugation/HAIN001_typeF
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN001/HAIN001.faa for system(s):
	- typeF
MacSyFinder's results will be stored in Conjugation/HAIN001_typeB
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN001/HAIN001.faa for system(s):
	- typeB
MacSyFinder's results will be stored in Conjugation/HAIN001_typeC
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN001/HAIN001.faa for system(s):
	- typeC
MacSyFinder's results will be stored in Conjugation/HAIN001_typeFATA
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN001/HAIN001.faa for system(s):
	- typeFATA
MacSyFinder's results will be stored in Conjugation/HAIN001_typeFA
Analysis la

MacSyFinder's results will be stored in Conjugation/HAIN002_typeI
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN002/HAIN002.faa for system(s):
	- typeI

************************************
 Analyzing clusters 
************************************


--- Cluster typeI  ---
['HAIN002_00115', 'HAIN002_00126', 'HAIN002_00159']
['t4cp2', 'virb4', 'MOBH']
[111, 122, 155]
Considering typeI: 
-> uncomplete
=> Putative typeI locus stored for later treatment of scattered systems.

    
****************************************************
******* Report scattered/uncompleted systems *******
****************************************************

******************************************

******************************************
Building reports of detected systems
 
Counter()
******************************************
MacSyFinder's results will be stored in Conjugation/HAIN002_typeT
Analysis launched on /data/ext4/dataD

MacSyFinder's results will be stored in Conjugation/HAIN006_typeFA
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN006/HAIN006.faa for system(s):
	- typeFA
MacSyFinder's results will be stored in Conjugation/HAIN006_typeG
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN006/HAIN006.faa for system(s):
	- typeG
MacSyFinder's results will be stored in Conjugation/HAIN006_typeI
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN006/HAIN006.faa for system(s):
	- typeI
MacSyFinder's results will be stored in Conjugation/HAIN006_typeT
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN006/HAIN006.faa for system(s):
	- typeT
MacSyFinder's results will be stored in Conjugation/HAIN007_typeF
Analysis launche


************************************
 Analyzing clusters 
************************************


--- Cluster typeT  ---
['HAIN007_00104', 'HAIN007_00115', 'HAIN007_00148']
['t4cp2', 'virb4', 'MOBH']
[100, 111, 144]
Considering typeT: 
-> uncomplete
=> Putative typeT locus stored for later treatment of scattered systems.

    
****************************************************
******* Report scattered/uncompleted systems *******
****************************************************

******************************************

******************************************
Building reports of detected systems
 
Counter()
******************************************
MacSyFinder's results will be stored in Conjugation/HAIN008_typeF
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN008/HAIN008.faa for system(s):
	- typeF
MacSyFinder's results will be stored in Conjugation/HAIN008_typeB
Analysis launched on /data/ext4/dataD

MacSyFinder's results will be stored in Conjugation/HAIN009_typeI
Analysis launched on /data/ext4/dataDP/dev/fix_macsynfider/Macsyfinder_models/models/Conjugation/Sequences/Annotations/HAIN009/HAIN009.faa for system(s):
	- typeI

************************************
 Analyzing clusters 
************************************


--- Cluster typeI  ---
['HAIN009_00556', 'HAIN009_00582', 'HAIN009_00593']
['MOBH', 'virb4', 't4cp2']
[527, 553, 564]
Considering typeI: 
-> uncomplete
=> Putative typeI locus stored for later treatment of scattered systems.

    
****************************************************
******* Report scattered/uncompleted systems *******
****************************************************

******************************************

******************************************
Building reports of detected systems
 
Counter()
******************************************
MacSyFinder's results will be stored in Conjugation/HAIN009_typeT
Analysis launched on /data/ext4/dataD

In [44]:
!wc Conjugation/*/macsyfinder.report

    0     0     0 Conjugation/HAIN002_typeB/macsyfinder.report
    0     0     0 Conjugation/HAIN002_typeC/macsyfinder.report
    0     0     0 Conjugation/HAIN002_typeF/macsyfinder.report
    0     0     0 Conjugation/HAIN002_typeFA/macsyfinder.report
    0     0     0 Conjugation/HAIN002_typeFATA/macsyfinder.report
   22   352  3355 Conjugation/HAIN002_typeG/macsyfinder.report
    0     0     0 Conjugation/HAIN002_typeI/macsyfinder.report
    0     0     0 Conjugation/HAIN002_typeT/macsyfinder.report
    0     0     0 Conjugation/HAIN007_typeB/macsyfinder.report
    0     0     0 Conjugation/HAIN007_typeC/macsyfinder.report
    0     0     0 Conjugation/HAIN007_typeF/macsyfinder.report
    0     0     0 Conjugation/HAIN007_typeFA/macsyfinder.report
    0     0     0 Conjugation/HAIN007_typeFATA/macsyfinder.report
   22   352  3352 Conjugation/HAIN007_typeG/macsyfinder.report
    0     0     0 Conjugation/HAIN007_typeI/macsyfinder.report
    0     0     0 Conjugation/HAIN007_typeT/mac

We have 3 ICEs of type G in strain 2, 7 and 9

In [45]:
!awk 'NR==1||FNR>1' Conjugation/*/macsyfinder.report > Conj_results

# Identify the flanking core genes: finding a spot

In [46]:
import pandas as pd
import numpy as np

In [48]:
# deprecated # conj = pd.read_table("Conj_results", sep='\t')
conj = pd.read_csv("Conj_results", sep='\t')

In [49]:
core = pd.read_csv("core_genome_2.txt", sep='\t', header=None, names=["HAIN{:03d}".format(i) for i in range(1,11)])

In [52]:
conj.head()

,#Hit_Id,Replicon_name,Position,Sequence_length,Gene,Reference_system,Predicted_system,System_Id,System_status,Gene_status,i-evalue,Score,Profile_coverage,Sequence_coverage,Begin_match,End_match
0,HAIN002_00111,UserReplicon,107,209,G_tfc2,typeG,typeG,UserReplicon_typeG_1,single_locus,accessory,2.600000e-60,200.1,0.921,0.990,2,208
1,HAIN002_00112,UserReplicon,108,246,G_tfc3,typeG,typeG,UserReplicon_typeG_1,single_locus,accessory,5.800000e-88,290.7,0.975,0.972,7,245
2,HAIN002_00114,UserReplicon,110,168,G_tfc5,typeG,typeG,UserReplicon_typeG_1,single_locus,accessory,6.600000e-56,185.4,0.983,0.994,2,168
3,HAIN002_00115,UserReplicon,111,749,t4cp2,CONJ,typeG,UserReplicon_typeG_1,single_locus,mandatory,6.000000e-35,117.3,0.991,0.330,404,650
4,HAIN002_00117,UserReplicon,113,228,G_tfc7,typeG,typeG,UserReplicon_typeG_1,single_locus,accessory,8.400000e-85,281.1,0.972,0.991,3,228


Reformat Macsyfinder output, replace UserReplicon by the name of the replicon. (I may have missed an option)

In [54]:
# Attention code fragile, va chercher les 7 premiers caracters de la colonne 1
conj["Replicon_name"] = conj["#Hit_Id"].apply(lambda x: x[:7])

In [55]:
conj.head()

,#Hit_Id,Replicon_name,Position,Sequence_length,Gene,Reference_system,Predicted_system,System_Id,System_status,Gene_status,i-evalue,Score,Profile_coverage,Sequence_coverage,Begin_match,End_match
0,HAIN002_00111,HAIN002,107,209,G_tfc2,typeG,typeG,UserReplicon_typeG_1,single_locus,accessory,2.600000e-60,200.1,0.921,0.990,2,208
1,HAIN002_00112,HAIN002,108,246,G_tfc3,typeG,typeG,UserReplicon_typeG_1,single_locus,accessory,5.800000e-88,290.7,0.975,0.972,7,245
2,HAIN002_00114,HAIN002,110,168,G_tfc5,typeG,typeG,UserReplicon_typeG_1,single_locus,accessory,6.600000e-56,185.4,0.983,0.994,2,168
3,HAIN002_00115,HAIN002,111,749,t4cp2,CONJ,typeG,UserReplicon_typeG_1,single_locus,mandatory,6.000000e-35,117.3,0.991,0.330,404,650
4,HAIN002_00117,HAIN002,113,228,G_tfc7,typeG,typeG,UserReplicon_typeG_1,single_locus,accessory,8.400000e-85,281.1,0.972,0.991,3,228


In [57]:
conj.System_Id.replace("UserReplicon", conj.Replicon_name, regex=True, inplace=True)

In [58]:
conj.head()

,#Hit_Id,Replicon_name,Position,Sequence_length,Gene,Reference_system,Predicted_system,System_Id,System_status,Gene_status,i-evalue,Score,Profile_coverage,Sequence_coverage,Begin_match,End_match
0,HAIN002_00111,HAIN002,107,209,G_tfc2,typeG,typeG,HAIN002_typeG_1,single_locus,accessory,2.600000e-60,200.1,0.921,0.990,2,208
1,HAIN002_00112,HAIN002,108,246,G_tfc3,typeG,typeG,HAIN002_typeG_1,single_locus,accessory,5.800000e-88,290.7,0.975,0.972,7,245
2,HAIN002_00114,HAIN002,110,168,G_tfc5,typeG,typeG,HAIN002_typeG_1,single_locus,accessory,6.600000e-56,185.4,0.983,0.994,2,168
3,HAIN002_00115,HAIN002,111,749,t4cp2,CONJ,typeG,HAIN002_typeG_1,single_locus,mandatory,6.000000e-35,117.3,0.991,0.330,404,650
4,HAIN002_00117,HAIN002,113,228,G_tfc7,typeG,typeG,HAIN002_typeG_1,single_locus,accessory,8.400000e-85,281.1,0.972,0.991,3,228


In [53]:
core.head()

,HAIN001,HAIN002,HAIN003,HAIN004,HAIN005,HAIN006,HAIN007,HAIN008,HAIN009,HAIN010
0,HAIN001_00203,HAIN002_00273,HAIN003_00473,HAIN004_00707,HAIN005_01972,HAIN006_01111,HAIN007_00257,HAIN008_00454,HAIN009_00399,HAIN010_00871
1,HAIN001_01230,HAIN002_01260,HAIN003_01252,HAIN004_01896,HAIN005_01094,HAIN006_01885,HAIN007_01365,HAIN008_01198,HAIN009_01245,HAIN010_01455
2,HAIN001_00104,HAIN002_00172,HAIN003_00582,HAIN004_00596,HAIN005_00137,HAIN006_00099,HAIN007_00161,HAIN008_00553,HAIN009_00503,HAIN010_01444
3,HAIN001_00588,HAIN002_00655,HAIN003_00034,HAIN004_01214,HAIN005_01729,HAIN006_00655,HAIN007_00715,HAIN008_00024,HAIN009_00024,HAIN010_00204
4,HAIN001_00677,HAIN002_00682,HAIN003_01821,HAIN004_01240,HAIN005_01625,HAIN006_00556,HAIN007_00803,HAIN008_01742,HAIN009_01872,HAIN010_00228


To identify the flanking core genes, I sort the column in the table `core` corresponding to the genome where I have an ICE, and find the index where I would insert any genes of that ICE

On va trier le tableau core genome en utilisant 3 colonnes (les replicons HAIN002, HAIN007 et HAIN009)

In [73]:
flanking_core = {}
for ice in conj.groupby("System_Id"):
    print 'dealing with', ice[0], 'which is a', type(ice)
    data = ice[1] # since ice[0] is just the name of the system_Id i.e. HAIN002_typeG_1
    replicon = data.Replicon_name.unique()[0]
    
    print 'sorting rows according to', replicon
    # sort the column in the table core corresponding to the genome where I have an ICE
    core.sort_values(replicon, inplace=True)
    #print (spots[0])
    
    spots = np.unique([core[replicon].searchsorted(gene) for gene in data["#Hit_Id"]])
    print('spot:', spots)
    assert len(spots)==1 # make sure that all genes of the conjugative system insert at the same spot
    flanking_core[ice[0]] = core.iloc[spots[0]-1:spots[0]+1].index
    

dealing with HAIN002_typeG_1 which is a <type 'tuple'>
sorting rows according to HAIN002
('spot:', array([50]))
dealing with HAIN007_typeG_1 which is a <type 'tuple'>
sorting rows according to HAIN007
('spot:', array([50]))
dealing with HAIN009_typeG_1 which is a <type 'tuple'>
sorting rows according to HAIN009
('spot:', array([345]))


In [74]:
flanking_core

{'HAIN002_typeG_1': Int64Index([267, 162], dtype='int64'),
 'HAIN007_typeG_1': Int64Index([267, 162], dtype='int64'),
 'HAIN009_typeG_1': Int64Index([162, 267], dtype='int64')}

In [75]:
core.sort_index(inplace=True) # reset the order of the table 

In [76]:
flanking_core

{'HAIN002_typeG_1': Int64Index([267, 162], dtype='int64'),
 'HAIN007_typeG_1': Int64Index([267, 162], dtype='int64'),
 'HAIN009_typeG_1': Int64Index([162, 267], dtype='int64')}

In [77]:
flanking_core_ID = core.loc[flanking_core.values()[0]]

N.B l'index du tableau est a 0

In [79]:
flanking_core_ID

,HAIN001,HAIN002,HAIN003,HAIN004,HAIN005,HAIN006,HAIN007,HAIN008,HAIN009,HAIN010
267,HAIN001_00088,HAIN002_00089,HAIN003_00595,HAIN004_00583,HAIN005_00080,HAIN006_00083,HAIN007_00079,HAIN008_00566,HAIN009_00619,HAIN010_00854
162,HAIN001_00093,HAIN002_00162,HAIN003_00590,HAIN004_00589,HAIN005_00130,HAIN006_00088,HAIN007_00151,HAIN008_00561,HAIN009_00513,HAIN010_00849


We see that they are all in the same spot, inbetween core genes 221 and 783, and that strain number 9 has been sequenced on the complementary strand or has a synteny breakpoint at that position. Let's eliminate that latter possibility.

There is a breakpoint if the two flanking coregenes in the list of core genes are not continguous (in case of a inversion of one of the two coregenes with a third one for instance). Meaning that the difference between their rank is either 1, -1 or the number of coregenes (-1) (to wrap around).   

Getting their rank

In [80]:
order = core.values.argsort(axis=0)
ranks = order.argsort(axis=0)

In [81]:
for k,v in flanking_core.iteritems():
    
    rank_diff = np.abs(np.diff(ranks[[v[0], v[1]]], axis=0)[0])
    bkp = np.where((rank_diff!=1) & (rank_diff!=len(core)-1))[0]
    if len(bkp)>0:
        end_sentence = "is breakpoint for strain {}".format(bkp)
    else:
        end_sentence = "is no breakpoint in any strain"
    print "The ICE {} is in the spot flanked by coregenes {} and {} and there {}".format(k, v[0], v[1], end_sentence)

The ICE HAIN007_typeG_1 is in the spot flanked by coregenes 267 and 162 and there is no breakpoint in any strain
The ICE HAIN009_typeG_1 is in the spot flanked by coregenes 162 and 267 and there is no breakpoint in any strain
The ICE HAIN002_typeG_1 is in the spot flanked by coregenes 267 and 162 and there is no breakpoint in any strain


Here there is no breakpoint in any the strains at that spot. In case there is a breakpoint, one should remove the corresponding strain of the analysis. 

Let's get the information on the relative sequencing orientation so we can normalize the strand position of the genes. We fix the reference with respect to the strain 2 (arbitrary, the first strain with an ICE)

In [82]:
sequencing_orientation = {i:j for i,j in zip(core.columns, np.median(np.diff(ranks[ranks[:,1].argsort()], axis=0), axis=0))}

In [83]:
sequencing_orientation

{'HAIN001': 1.0,
 'HAIN002': 1.0,
 'HAIN003': -1.0,
 'HAIN004': 1.0,
 'HAIN005': -1.0,
 'HAIN006': -1.0,
 'HAIN007': 1.0,
 'HAIN008': -1.0,
 'HAIN009': -1.0,
 'HAIN010': 1.0}

So the first two strain and the 7th and 10th are in the same median orientation, other strain will need to be flipped. We took the median orientation so it's not garanteed that it will work, as the ICE might be in a bigger region that have been inversed

# Pan-Genome of the spot

We now have the 2 flanking core genes. We'll gather all the genes that are in the spot and cluster them with uclust to make gene families.

In [85]:
np.unique(flanking_core.values())

array([162, 267])

metI: HAIN001_00088	HAIN002_00089	HAIN003_00595	HAIN004_00583	HAIN005_00080	HAIN006_00083	HAIN007_00079	HAIN008_00566	HAIN009_00619	HAIN010_00854

thrC: HAIN001_00093	HAIN002_00162	HAIN003_00590	HAIN004_00589	HAIN005_00130	HAIN006_00088	HAIN007_00151	HAIN008_00561	HAIN009_00513	HAIN010_00849



In [86]:
good_core = core.loc[np.unique(flanking_core.values())]
good_core

,HAIN001,HAIN002,HAIN003,HAIN004,HAIN005,HAIN006,HAIN007,HAIN008,HAIN009,HAIN010
162,HAIN001_00093,HAIN002_00162,HAIN003_00590,HAIN004_00589,HAIN005_00130,HAIN006_00088,HAIN007_00151,HAIN008_00561,HAIN009_00513,HAIN010_00849
267,HAIN001_00088,HAIN002_00089,HAIN003_00595,HAIN004_00583,HAIN005_00080,HAIN006_00083,HAIN007_00079,HAIN008_00566,HAIN009_00619,HAIN010_00854


Make list of gene for each interval 

In [90]:
mkdir Spots

In [103]:
strains = []
for strain in good_core.columns:
    strains.append(strain)
    i = [v.strip() for v in good_core[strain].values]
    f,l = int(i[0][-5:]), int(i[1][-5:])
    s = SeqIO.index("Sequences/Annotations/{strain}/{strain}.faa".format(strain=strain), "fasta")
    seqs = [s.get("{}{:05d}".format(i[0][:8], j)) for j in range(min(f,l), max(f,l)+1)]
    SeqIO.write([seq for seq in seqs if seq is not None],
                "Spots/{}_spot.faa".format(strain),
                "fasta")

#### Here we only have 1 spot, but one should keep track of the spots

In [92]:
!grep -c ">" Spots/*faa

Spots/HAIN001_spot.faa:2
Spots/HAIN002_spot.faa:70
Spots/HAIN003_spot.faa:2
Spots/HAIN004_spot.faa:3
Spots/HAIN005_spot.faa:46
Spots/HAIN006_spot.faa:2
Spots/HAIN007_spot.faa:69
Spots/HAIN008_spot.faa:2
Spots/HAIN009_spot.faa:102
Spots/HAIN010_spot.faa:2


In [93]:
!cat Spots/*faa > Spots/All_prot_spot.prt

In [94]:
mkdir Spots/Clusters

In [95]:
!usearch -quiet -cluster_fast Spots/All_prot_spot.prt \
         -id 0.7\
         -centroids Spots/All_prot_spot-70.fasta \
         -uc Spots/All_prot_spot-70.uc > Spots/log_usearch


In [96]:
cluster = pd.read_csv("Spots/All_prot_spot-70.uc", sep='\t', header=None)

In [97]:
cluster

,0,1,2,3,4,5,6,7,8,9
0,S,0,172,*,.,*,*,*,HAIN002_00102 hypothetical protein,*
1,H,0,172,100.0,.,0,172,=,HAIN007_00092 hypothetical protein,HAIN002_00102 hypothetical protein
2,H,0,172,100.0,.,0,172,=,HAIN009_00606 hypothetical protein,HAIN002_00102 hypothetical protein
3,S,1,156,*,.,*,*,*,HAIN002_00108 hypothetical protein,*
4,H,1,156,100.0,.,0,156,=,HAIN009_00600 hypothetical protein,HAIN002_00108 hypothetical protein
5,S,2,198,*,.,*,*,*,HAIN002_00109 hypothetical protein,*
6,H,2,198,100.0,.,0,198,=,HAIN009_00599 hypothetical protein,HAIN002_00109 hypothetical protein
7,S,3,209,*,.,*,*,*,HAIN002_00111 hypothetical protein,*
8,H,3,209,100.0,.,0,209,=,HAIN009_00597 hypothetical protein,HAIN002_00111 hypothetical protein
9,S,4,168,*,.,*,*,*,HAIN002_00114 hypothetical protein,*


In [98]:
families = cluster[cluster[0]!="C"].groupby(1)[8].apply(lambda x: sorted(x)).values

In [170]:
fam = pd.DataFrame([sorted(f) for f in families])

In [171]:
fam.sort_values(by=fam.columns.tolist(), inplace=True)
fam.sort_values(by=fam.index.tolist(), axis=1, inplace=True)

In [172]:
fam.head()

,0,1,2,3,4,5,6,7,8,9
30,HAIN001_00088 Cystathionine gamma-synthase/O-a...,HAIN002_00089 Cystathionine gamma-synthase/O-a...,HAIN003_00595 Cystathionine gamma-synthase/O-a...,HAIN004_00583 Cystathionine gamma-synthase/O-a...,HAIN005_00080 Cystathionine gamma-synthase/O-a...,HAIN006_00083 Cystathionine gamma-synthase/O-a...,HAIN007_00079 Cystathionine gamma-synthase/O-a...,HAIN008_00566 Cystathionine gamma-synthase/O-a...,HAIN009_00619 Cystathionine gamma-synthase/O-a...,HAIN010_00854 Cystathionine gamma-synthase/O-a...
19,HAIN001_00093 Threonine synthase,HAIN002_00162 Threonine synthase,HAIN003_00590 Threonine synthase,HAIN004_00589 Threonine synthase,HAIN005_00130 Threonine synthase,HAIN006_00088 Threonine synthase,HAIN007_00151 Threonine synthase,HAIN008_00561 Threonine synthase,HAIN009_00513 Threonine synthase,HAIN010_00849 Threonine synthase
100,HAIN002_00093 hypothetical protein,HAIN007_00083 hypothetical protein,HAIN009_00615 hypothetical protein,None,None,None,None,None,None,None
20,HAIN002_00094 replicative DNA helicase,HAIN007_00084 replicative DNA helicase,HAIN009_00614 replicative DNA helicase,None,None,None,None,None,None,None
60,HAIN002_00095 hypothetical protein,HAIN007_00085 hypothetical protein,HAIN009_00613 hypothetical protein,None,None,None,None,None,None,None


In [173]:
# The next cells are extra code to correct a major bug
fam2 = fam  # Copy the panda dataframe

# Adjust the column names by simply assign it to the .columns attribute:
fam2.columns = strains

In [174]:
# Note the Families names, this info should disappear
fam2.head()

,HAIN001,HAIN002,HAIN003,HAIN004,HAIN005,HAIN006,HAIN007,HAIN008,HAIN009,HAIN010
30,HAIN001_00088 Cystathionine gamma-synthase/O-a...,HAIN002_00089 Cystathionine gamma-synthase/O-a...,HAIN003_00595 Cystathionine gamma-synthase/O-a...,HAIN004_00583 Cystathionine gamma-synthase/O-a...,HAIN005_00080 Cystathionine gamma-synthase/O-a...,HAIN006_00083 Cystathionine gamma-synthase/O-a...,HAIN007_00079 Cystathionine gamma-synthase/O-a...,HAIN008_00566 Cystathionine gamma-synthase/O-a...,HAIN009_00619 Cystathionine gamma-synthase/O-a...,HAIN010_00854 Cystathionine gamma-synthase/O-a...
19,HAIN001_00093 Threonine synthase,HAIN002_00162 Threonine synthase,HAIN003_00590 Threonine synthase,HAIN004_00589 Threonine synthase,HAIN005_00130 Threonine synthase,HAIN006_00088 Threonine synthase,HAIN007_00151 Threonine synthase,HAIN008_00561 Threonine synthase,HAIN009_00513 Threonine synthase,HAIN010_00849 Threonine synthase
100,HAIN002_00093 hypothetical protein,HAIN007_00083 hypothetical protein,HAIN009_00615 hypothetical protein,None,None,None,None,None,None,None
20,HAIN002_00094 replicative DNA helicase,HAIN007_00084 replicative DNA helicase,HAIN009_00614 replicative DNA helicase,None,None,None,None,None,None,None
60,HAIN002_00095 hypothetical protein,HAIN007_00085 hypothetical protein,HAIN009_00613 hypothetical protein,None,None,None,None,None,None,None


In [176]:
# Voici la commande qui corrige le bug
# On itere sur le dataframe (par colonne) et on remplace une expression regulieres dans chacune des rangees
for i in strains:
    fam2[i]= fam2[i].str.replace(r'(\S+_\d+)(.+)', r"\1")

In [177]:
fam2.head()

,HAIN001,HAIN002,HAIN003,HAIN004,HAIN005,HAIN006,HAIN007,HAIN008,HAIN009,HAIN010
30,HAIN001_00088,HAIN002_00089,HAIN003_00595,HAIN004_00583,HAIN005_00080,HAIN006_00083,HAIN007_00079,HAIN008_00566,HAIN009_00619,HAIN010_00854
19,HAIN001_00093,HAIN002_00162,HAIN003_00590,HAIN004_00589,HAIN005_00130,HAIN006_00088,HAIN007_00151,HAIN008_00561,HAIN009_00513,HAIN010_00849
100,HAIN002_00093,HAIN007_00083,HAIN009_00615,None,None,None,None,None,None,None
20,HAIN002_00094,HAIN007_00084,HAIN009_00614,None,None,None,None,None,None,None
60,HAIN002_00095,HAIN007_00085,HAIN009_00613,None,None,None,None,None,None,None


# Format data to plot 

We use and old script I wrote which is quite ugly to read, but works. You can find it here: [spot_ICE](https://gitlab.pasteur.fr/gem/spot_ICE)

The advantage is that it produces an interactive plot ot the spot, but the drawback is that the input format is a bit weird. Let's get through


In [164]:
from Bio import SeqUtils

The input file of the script looks like:

    #header
    \n
    ICE_ID
    genome1&2 genome2&2 genome3&50 genome4&2 #list of genome ID with the number of gene in between the 2 core genes (included)
    <Table with genes from the genomes with the ICE in columns, see below for description>
    0.4 0.45 0.43 .... #GC% values along the spot.
    \n
Where the table has the following columns:

|Column|description|Value| Comment|
|-:|:-|:-|:-:|
|1| Gene_id | String ||
|2| Annotation| String ||
|3| Start| Integer ||
|4| End| Integer||
|5| Gene orientation | 1/- 1||
|6| Replicon orientation | 1/-1 | compare to the other genome of the species, is this replicon sequenced on the same strand or not ? (same average orientation of the core genes)|
|7| Normalized orientation | (Gene orientation * Replicon orientation)||
|8| Gene size| Integer||
|9| MPF profile | MOBH/G_tfc23/...| if the gene is matched by macsyfinder, which profile matched ?|
|10| Integrase_type | Tyr_rec | if the gene is matched by tyrosine or serine recombinase profiles|
|11| Gene families | HAIN002_00160&HAIN009_00555 | genes in other genome of the species that are homologous.|
|12| RNA |tRNA&tRNA-Lys&143658&143734&1 | RFAM hits, in the line of the closest gene, and formatted as: RNA_type&RFAM_number&start&end&strand|

In any case, if there is not value, it must be a 0

#### ...

In [179]:
# Slightly edited by JSB to ensure that gene_fams infos look like:
        # HAIN002_00089    HAIN001_00088&HAIN003_00595&HAIN004_00583&HAIN...
        # HAIN002_00093                          HAIN007_00083&HAIN009_00615
        # HAIN002_00094                          HAIN007_00084&HAIN009_00614
        # HAIN002_00095                          HAIN007_00085&HAIN009_00613
        # HAIN002_00096                          HAIN007_00086&HAIN009_00612

# The code below use fam2 panda dataframe rather than fam           

with open("All_data_hain.txt", "w") as outf:
    outf.write("#Annotation\tgene_name\tStart\tEnd\tStrand\tStrand_replicon\tStrand_norm\tSize\tGene\tIntegrase\tgene_fam\tRNA\n\n") # Header optional, could be an empty line

    for sys_ID in conj.System_Id.unique():
        df_sys_ID = pd.DataFrame(columns=fam2.columns[:-1])
        strain = sys_ID.split("_")[0]
        for f in fam2.iterrows():
            try:
                idx = f[1][f[1].str.contains(strain, na=False)].index
                df_sys_ID.loc[f[1][idx[0]]] = f[1][f[1].index.isin(idx)==0].values
            except IndexError:
                pass
        df_sys_ID.sort_index(inplace=True)
        # spot_size is like: ['HAIN001&2', 'HAIN002&70', 'HAIN003&2', 'HAIN004&3', 'HAIN005&46', ...]
        spot_size = ["&".join((m, str(n))) for m,n in pd.Series([i.split("_")[0].strip() for i in fam2.values.ravel() if isinstance(i, str)]).value_counts().sort_index().iteritems()]
        gbk = SeqIO.read("Sequences/Annotations/{strain}/{strain}.gbk".format(strain=strain), "genbank")

        final = pd.DataFrame(columns=["Annotation", "gene_name", "Start", "End", "Strand"])
        for feat in gbk.features[1:]:
            if min(flanking_core_ID[strain]) <= feat.qualifiers["locus_tag"][0] <= max(flanking_core_ID[strain]):
                if "gene" in feat.qualifiers:
                    gene_name = feat.qualifiers["gene"][0]
                elif feat.type=="tRNA":
                    gene_name = feat.qualifiers["note"][0]
                else:
                    gene_name = "HP"
                final.loc[feat.qualifiers["locus_tag"][0]] = feat.qualifiers["product"][0], gene_name, feat.location.start, feat.location.end, feat.location.strand

        final["Strand_replicon"] = sequencing_orientation[strain]
        final["Strand_norm"] = final.Strand*final.Strand_replicon
        final["Size"] = final.End - final.Start + 1
        final = final.merge(conj.set_index("#Hit_Id")[["Gene"]], left_index=True, right_index=True, how="left")
        final["Integrase"] = np.nan # If you detected integrases you can put something else than Nan or 0 in front of the corresponding gene.
        gene_fams = df_sys_ID.apply(lambda x: "&".join(x.dropna()), axis=1).replace("", "0")
        
        # gene_fams is like:
        # HAIN002_00089    HAIN001_00088&HAIN003_00595&HAIN004_00583&HAIN...
        # HAIN002_00093                          HAIN007_00083&HAIN009_00615
        # HAIN002_00094                          HAIN007_00084&HAIN009_00614
        # HAIN002_00095                          HAIN007_00085&HAIN009_00613
        # HAIN002_00096                          HAIN007_00086&HAIN009_00612
             
        final = final.merge(gene_fams.to_frame(name="gene_fam"), left_index=True, right_index=True, how='left')
        final["RNA"] = final.apply(lambda x: "&".join(("tRNA",
                                                       x["Annotation"],
                                                       str(int(x["Start"])),
                                                       str(int(x["End"])),
                                                       str(int(x["Strand_norm"])))) if "tRNA" in x["Annotation"] else np.nan, axis=1)

        final.fillna(0, inplace=True)
        final[final.select_dtypes([float]).columns] = final.select_dtypes([float]).astype(int, inplace=1) 
        GC = [SeqUtils.GC(gbk.seq[final.Start[0]:final.End[-1]][i:i+1000]) for i in range(0, final.End[-1]-final.Start[0]+1, 1000)]
        outf.write(sys_ID+"\n")
        outf.write(" ".join(spot_size)+"\n")
        final.to_csv(outf, header=None, sep="\t")
        outf.write(" ".join(map(str, GC))+"\n\n")
print "Done"

Done


# Plot the spot

Once you downloaded the script provided earlier, just type (in a terminal preferably, and ***not*** in a interactive python environnenment (note the *!* at the beginning of the line, call a shell)

The window can be clicked:

- left click on gene: print information for that genes
- left click on any x-axis: erase all gene informations
- right click on 2 genes in the same ICE: select the gene as a first and last genes delimiting the ICE
- shift + right-click: remove the selected genes in the corresponding ICE.
    - Beware to click 2 times in the same ICE otherwise the limits will be mixed between ICEs (in case of odd number of right-clicks).
    - If you click 4 times in the same ICE, it will define 2 ICEs in tandem. (Or any even number of right click will define half as many ICE)
- The spot are plotted 4 at a times, and the window close every 4. You can increase this number in the script if you have a big enough screen. 
- You can export an ICE in its spot that you'd like by ticking the box on the right (weird rectangles) and click export. It will create a file `Selection...` that you can re-use. You could easily merge 2 exported files from different window. 

In [ ]:
# Rouler le script dans un terminal ordinaire 
#./plot_ICE_spot.py All_data_hain.txt

Here is a screenshot, obtained with this entire pipeline:
<img src="Screenshot_spot_ICE.png" alt="spot_ICE" style="width: 75%;"/>
